# DREAM instrument view

This notebook is a simple example on how to use the instrument view for the DREAM instrument.

- The DREAM-specific instrument view is capable of slicing the data with a slider widget along a dimension (e.g. `tof`) by using the `dim` argument.
- There are also checkboxes to hide/show the different modules that make up the DREAM detectors.

In [ ]:
import pandas as pd
import scipp as sc
from ess import dream

## Load and reshape the data

We load a data set from a Geant4 simulation (stored as a `.csv` file).
After loading, we manipulate and reshape the data to obtain a data group that contains one entry per detector bank.
In each detector bank, the data is organised by `wire`, `strip`, `module`, `segment`, and `counter`.

It is assumed that Nexus files from the DREAM beamline will be loaded as such by Scippnexus,
and that the manipulations required below would not be needed for production datasets.

In [ ]:
df = pd.read_table(dream.data.get_path('data_dream_HF_mil_closed_alldets_1e9.csv.zip'))
ds = sc.compat.from_pandas(df)
# Some logic to parse units in the file header
for key in list(ds):
    name, *remainder = key.split(' ')
    ds[name] = ds.pop(key)
    ds[name].unit = remainder[0][1:-1] if remainder else None
ds['lambda'].unit = 'angstrom'
table = sc.DataArray(sc.ones(sizes=ds.sizes, unit='counts'))
for name in ds:
    table.coords[name] = ds[name].data
table.coords['tof'] = table.coords['tof'].to(dtype=float, unit='us')
# Group pixels according to their bank identifier
mapping = {'sumo0': 0, 'sumo1': 1, 'sumo2': 2, 'sumo3': 3, 'mantle': 4, 'high_res': 5}
da = table[::5].copy().group('det')  # Limit number of points rendered for docs size
dg = sc.DataGroup({key: da['det', val] for key, val in mapping.items()})
dg = sc.DataGroup(
    {
        key: da.group('module', 'segment', 'counter', 'wire', 'strip')
        for key, da in dg.items()
    }
)
for da in dg.values():
    da.coords['x'] = da.bins.coords.pop('x_pos').bins.mean()
    da.coords['y'] = da.bins.coords.pop('y_pos').bins.mean()
    da.coords['z'] = da.bins.coords.pop('z_pos').bins.mean()
dg

## Instrument view

We first histogram the data along the time-of-flight (`tof`) dimension,
making sure the same bins are used for all modules:

In [ ]:
tof_edges = sc.linspace('tof', 1.0e4, 1.0e5, 51, unit='us')
data = dg.hist(tof=tof_edges)

We now use the `instrument_view` function to show the 3D view of the instrument pixels,
specifying that we wish to have a slider along the `tof` dimension:

In [ ]:
full_view = dream.instrument_view(data, dim='tof')
full_view

In [ ]:
full_view[2].controls['tof']['slider'].value = 35

It is also possible to view a single detector module using

In [ ]:
mantle_view = dream.instrument_view(data['mantle'], dim='tof')
mantle_view

In [ ]:
mantle_view[1].controls['tof']['slider'].value = 35